In [1]:
import weaviate
import weaviate.classes.config as wvcc
from weaviate.classes.config import Property, DataType, Configure
import requests
import json

# Cloud Connection

In [10]:
#define a client 
client = weaviate.connect_to_wcs(
    cluster_url="https://ep9buqisqh2agxcqhaqja.c0.us-central1.gcp.weaviate.cloud",
    auth_credentials=weaviate.classes.init.Auth.api_key("cBbDQMWkOgPEzzQYg2megPVbhrZ28P6mH33k"),
)

#checking client status
def check_client(client):
    try:
        # check client is ready or not
        response = client.is_ready()
        if response:
            print("Weaviate client is ready.")
        else:
            print("Weaviate client is not ready.")
    except Exception as e:
        print(f"Error checking Weaviate client status: {e}")
        
    
check_client(client)

Weaviate client is ready.


# Collection making

In [25]:
# create a basic collection without victorizer

client.collections.create(
    "new_test_collection",
    vectorizer_config= Configure.Vectorizer.text2vec_huggingface(model = "sentence-transformers/all-MiniLM-L6-v2")
)

collection = client.collections.get('new_test_collection')
collection

# Insert data 

In [29]:
def insert_data(client, url = 'https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json'):
    # fetch data from the repo
    repo = requests.get(url)
    data = json.loads(repo.text)
    
    # prepare our data
    question_objs = []
    for i, d in enumerate(data):
        question_objs.append({
            "answer" : d['Answer'],
            "question" : d['Question'],
            "category" : d['Category']
        })
        
    # insert it and store
    questions = client.collections.get("new_test_collection")
    questions.data.insert_many(question_objs)
    
    
insert_data(client)

# Query to show the data 

In [11]:
def fetch_and_print_data(client):
    try:
        # Access the collection
        collection = client.collections.get("new_test_collection")
        
        response = collection.query.near_text(
            query="biology", 
            limit=2 
        )
        
        # Print the properties of the returned objects
        if response.objects:
            for obj in response.objects:
                print(obj.properties) 
        else:
            print("No results found.")

    finally:
        client.close() 
        
fetch_and_print_data(client)

{'answer': 'DNA', 'question': 'In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance', 'category': 'SCIENCE'}
{'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}


# Update objects

In [33]:
# get data details 
def get_all_objects(client, collection_name = 'new_test_collection'):
    try:
        #reconnect the client
        client.connect()
        
        #get the collection
        collection = client.collections.get(collection_name)
        
        #get all objects
        for item in collection.iterator():
            print(item.uuid, item.properties)
            
    finally:
        client.close()
        
        
get_all_objects(client)

40490e6f-f017-481b-bf28-27c2255a42bd {'answer': 'species', 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'}
4d8ebfba-9f6b-453a-a00d-ef1f2b881540 {'answer': 'Sound barrier', 'question': 'In 70-degree air, a plane traveling at about 1,130 feet per second breaks it', 'category': 'SCIENCE'}
6f14b0a3-f96c-4428-9b35-29a241b908f2 {'answer': 'DNA', 'question': 'In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance', 'category': 'SCIENCE'}
a2f7d5a6-cce5-4315-a2b7-86682dc45743 {'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
c7ccc1b4-3dc5-4842-9586-8648c5970678 {'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'}
cc2b1ec9-f26e-4008-bde2-b6c2c97c614c {'answer': 'the nose or snout', 'question': 'T

In [22]:
#basically update a property
client.connect()
    
#get the collection
question = client.collections.get("new_test_collection")
    
#define the id
uuid = 'edbc46a7-4afa-480c-af6d-079db1b356e6'
    
#update a property
question.data.update(
    uuid = uuid,
    properties={
        'answer' : 'species',
        'category' : 'ANIMALS'
    }
)

In [25]:
# update the properties
def update_property(client, uuid, collection_name = 'new_test_collection', properties_list = None, values_list = None):
    
    # prepare updating settings
    #check for properties and values lists existence
    if properties_list is None or values_list is None:
        print('you have to provide properties and values lists')
    else:
        pass
    
    #check two lists length
    if len(properties_list) != len(values_list):
        print('the length of two lists must be the same')
    else:
        pass
    
    # map roperties_list with values_list
    updated_data = dict(zip(properties_list, values_list))
    
    
    #prepare cloent and collection
    #reconnect the client
    client.connect()
    try:
        #get the collection
        question = client.collections.get(collection_name)
        
        #update the collection with updated date
        collection.data.update(
            uuid = uuid,
            properties = updated_data
        )
        
        print(f"Updated properties {updated_data} for object with UUID {uuid}.")
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        client.close()
        
        
update_property(
    client,
    uuid='edbc46a7-4afa-480c-af6d-079db1b356e6',
    collection_name='new_test_collection', 
    properties_list=['answer', 'category'],
    values_list=['an Atmospher', 'SCIENCE']
)

Updated properties {'answer': 'an Atmospher', 'category': 'SCIENCE'} for object with UUID edbc46a7-4afa-480c-af6d-079db1b356e6.


# Deleting an Object

In [37]:
from weaviate.classes.query import Filter

def delete_by_property(client, prop_name = None, prop_value = None, collection_name = 'new_test_collection'):
    
    #check for properties and values lists existence
    if prop_name is None or prop_value is None:
        print('you have to provide property and value')
    else:
        pass
    
    #reconnect the client
    client.connect()
    
    question = client.collections.get(collection_name)
    
    
    try:
        question.data.delete_many(
            where = Filter.by_property(prop_name).like(prop_value)
        )
    finally:
        client.close()

In [38]:
delete_by_property(client, 'answer', 'wire')